In [1]:
# model_checkpoint = "t5-small"
# model_checkpoint = "EleutherAI/pythia-160m"
model_checkpoint = "RWKV/rwkv-4-169m-pile"

In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("samsum")
print(raw_datasets)

/opt/conda/envs/llms/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating validation split: 100%|██████████| 818/818 [00:00<00:00, 218823.95 examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})


In [3]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    # if tokenizer.eos_token is not None:
    #     tokenizer.pad_token = tokenizer.eos_token
    # else:
    #     # If the tokenizer does not have an EOS token, you need to add a new pad token
    #     # This is an example, you may need to adjust it depending on the tokenizer
    #     tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:

def merge_fields(example):
        # 直接合并字符串
        example['text'] = example['dialogue'] + " TL;DR: " + example['summary']
        return example

transformed_dataset = raw_datasets.map(merge_fields)
transformed_dataset = transformed_dataset.remove_columns(raw_datasets['train'].column_names)
print(transformed_dataset)

print(transformed_dataset['train'][5])



# def process_data_to_model_inputs(examples):
#         model_inputs = tokenizer(examples["text"])

#         # Setup the tokenizer for targets
#         labels = tokenizer(text_target=examples["text"])
#         model_inputs["labels"] = labels["input_ids"]
#         return model_inputs

# def tokenize_function(examples):
#     return tokenizer(examples["text"])

# tokenized_datasets = transformed_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

# train_df = tokenized_datasets["train"].to_pandas()
# train_df["input_len"] = train_df["input_ids"].apply(len)
# print("train split:")
# print(train_df["input_len"].describe())
# median_length = train_df["input_len"].median()
# print("median:", median_length)


# print(tokenized_datasets)
# block_size = 128

# def group_texts(examples):
#     # # Concatenate all texts.
#     # concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
#     # total_length = len(concatenated_examples[list(examples.keys())[0]])
#     # # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
#     #     # customize this part to your needs.
#     # total_length = (total_length // block_size) * block_size
#     # # Split by chunks of max_len.
#     # result = {
#     #     k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
#     #     for k, t in concatenated_examples.items()
#     # }
#     # result["labels"] = result["input_ids"].copy()

#     examples["labels"] = examples["input_ids"].copy()
#     return examples

# # tokenized_datasets = tokenized_datasets.map(process_data_to_model_inputs, batched=True)

# tokenized_datasets = tokenized_datasets.map(
#     group_texts,
#     batched=True,
#     batch_size=1000,
#     num_proc=4,
# )


# # # Remove unnecessary columns
# # tokenized_datasets = tokenized_datasets.remove_columns(transformed_dataset['train'].column_names)

# print(tokenized_datasets)

Map: 100%|██████████| 818/818 [00:00<00:00, 17888.96 examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['text'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 818
    })
})
{'text': "Neville: Hi there, does anyone remember what date I got married on?\r\nDon: Are you serious?\r\nNeville: Dead serious. We're on vacation, and Tina's mad at me about something. I have a strange suspicion that this might have something to do with our wedding anniversary, but I have nowhere to check.\r\nWyatt: Hang on, I'll ask my wife.\r\nDon: Haha, someone's in a lot of trouble :D\r\nWyatt: September 17. I hope you remember the year ;) TL;DR: Wyatt reminds Neville his wedding anniversary is on the 17th of September. Neville's wife is upset and it might be because Neville forgot about their anniversary."}


In [5]:
from transformers import Trainer, AutoModelForCausalLM,TrainingArguments
from trl import SFTTrainer
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
model.resize_token_embeddings(len(tokenizer))
model.config.keys_to_ignore_at_inference = ["past_key_values","hidden_states", "attentions"]

In [6]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    # fp16=True,
    # push_to_hub=True,
)

In [7]:
trainer = SFTTrainer(
    model,
    args,
    train_dataset=transformed_dataset["train"],
    eval_dataset=transformed_dataset["validation"],
    tokenizer=tokenizer,
    dataset_text_field="text",
    max_seq_length=256 #padding的最大长度，这里256是数据集的长度的百分之75的四分位数
)

Map: 100%|██████████| 818/818 [00:00<00:00, 6595.73 examples/s]
/opt/conda/envs/llms/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [8]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.586100,2.529480


TrainOutput(global_step=921, training_loss=2.549298778289563, metrics={'train_runtime': 4886.6627, 'train_samples_per_second': 3.015, 'train_steps_per_second': 0.188, 'total_flos': 2954621536690176.0, 'train_loss': 2.549298778289563, 'epoch': 1.0})

In [9]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 12.55


实验记录：

EleutherAI/pythia-160m zero-shot: 23.15 Perplexity

EleutherAI/pythia-160m vinilla finetuning: 11.92 Perplexity （训练时间15分钟）gpu消耗6600

RWKV/rwkv-4-169m-pile zero-shot: 20.55 Perplexity

RWKV/rwkv-4-169m-pile vinilla finetuning: 12.55 Perplexity （训练时间一小时20分钟） gpu消耗10757